In [1]:
import os
import urllib.request
import zipfile
import tarfile

maximum_words = 500

def download_and_extract_if_not_exist(url, extract_path, zip_file_name, zip_file_contents_file_name):

    # If any of the files already exist, skip the download - it's already been done and extracted
    path = os.path.join(extract_path, zip_file_contents_file_name)
    if os.path.exists(path):
        print("Files already exist. Skipping download.")
        return

    # Check if the zip file already exists, if it does, skip the download to save time
    zip_file_path = os.path.join(extract_path, zip_file_name)
    if not os.path.exists(zip_file_path):
        print("Downloading zip file...")
        if not os.path.exists(extract_path):
            os.makedirs(extract_path)
        # Download the zip file
        zip_file_path, _ = urllib.request.urlretrieve(url, zip_file_path)
    else:
        print("Zip file already exists. Skipping download.")

    # Extract the contents of the zip file to the given directory based on the file extension
    if zip_file_path.endswith(".zip"):
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
    elif zip_file_path.endswith(".tar.gz") or zip_file_path.endswith(".tgz"):
        with tarfile.open(zip_file_path, 'r:gz') as tar_ref:
            tar_ref.extractall(extract_path)

    # Close and remove the zip file to free up space
    os.remove(zip_file_path)
    print(f"Files extracted to: {extract_path}")


In [2]:
# Download the word embeddings into the following folder:
folder_to_download = "./models"

# 6 Billion token model
embedding_url = "https://nlp.stanford.edu/data/glove.6B.zip"
file_name_to_download = "glove.6B.zip"

# 42 Billion token model
# embedding_url = "https://nlp.stanford.edu/data/wordvecs/glove.42B.300d.zip"
# file_name_to_download = "glove.42B.300d.zip"

# Download and extract the embedding file, then set a variable with the path to the embedding file
download_and_extract_if_not_exist(embedding_url, folder_to_download, file_name_to_download, "glove.6B.50d.txt")
word_embedding_file_path = os.path.join(folder_to_download, "glove.6B.50d.txt")

Files already exist. Skipping download.


In [3]:
# Download the dataset into the following folder:
folder_to_download = "./dataset"

# 6 Billion token model
dataset_url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
file_name_to_download = "aclImdb_v1.tar.gz"

# Download and extract, then set the path to the dataset
download_and_extract_if_not_exist(dataset_url, folder_to_download, file_name_to_download, "aclImdb/imdb.vocab")
dataset_folder= os.path.join(folder_to_download, "aclImdb")

Files already exist. Skipping download.


In [4]:
dataset_folder= os.path.join(folder_to_download, "aclImdb")
def read_imdb_data_from_folder(imdb_reviews_folder):
    texts = []
    labels = []
    review_folders = ["pos", "neg"] # there are 2 folders in each dataset folder: pos and neg
    for connotation_dir in review_folders:
        review_dir = os.path.join(imdb_reviews_folder, connotation_dir)
        for filename in os.listdir(review_dir):
            with open(os.path.join(review_dir, filename), "r", encoding="utf8") as f:
                texts.append(f.read())
            labels.append(0 if connotation_dir == "neg" else 1)
    return texts, labels

def get_negative_positive_from_label(label):
    return "negative" if label == 0 else "positive"

In [5]:
# Load the IMDB Movie Review dataset from the downloaded folder
test_texts, test_labels = read_imdb_data_from_folder(os.path.join(dataset_folder, "test"))
train_texts, train_labels = read_imdb_data_from_folder(os.path.join(dataset_folder, "train"))

In [6]:
for i in range(5):
    print(f"Review #{i}")
    print(f"Label: {train_labels[i+24000]}")
    print(f"Sentiment: {get_negative_positive_from_label(train_labels[i+24000])}")
    print(f"Text: {train_texts[i+24000]}")
    print()

Review #0
Label: 0
Sentiment: negative
Text: This wilfully bizarre adaptation of Borges short story is typical Cox. His strong visual sense is, as usual, undone by the appalling half baked acting of most of the cast. The film is definitely in the surreal tradition of Bunuel's Mexican period, and looks at times like a poor man's take on Lars Von Trier's Elements of Crime. Cox's apparent preference for single takes, jump cuts, and ambient sound recording all work against the film's effectiveness. Worth a look but ultimately disappointing.

Review #1
Label: 0
Sentiment: negative
Text: Lets first start this review with the fact that I SIGNED UP JUST TO WRITE THE REVIEW AND WARN PEOPLE TO SAVE Their MONEY!!<br /><br />This was one of the worst pieces of trash i have seen since The Hulk. The storyline was the most predictable garbage you could possibly come up with. If you are expecting 24 but on the big screen, flush that expectation down the toilet immediately along with the money you woul

In [7]:
import spacy

# Tokenize the sentences into words
train_sentence_example = train_texts[i+24000]

# Load the English Spacy Tokenizer Model
nlp = spacy.load("en_core_web_sm")
doc = nlp.tokenizer(train_sentence_example)

tokenized_test_text_docs = [nlp.tokenizer(text) for text in test_texts]
tokenized_train_text_docs = [nlp.tokenizer(text) for text in train_texts]


In [8]:
# Calculate the total length and count of nested arrays
total_length = 0
count = 0

for nested_array in tokenized_train_text_docs:
    total_length += len(nested_array)
    count += 1

# Calculate the average length
average_length = total_length / count
print(average_length)

272.46188


In [9]:
print_doc = tokenized_test_text_docs[24001]
print(print_doc)
for token in print_doc:
    print("")
    print(token)


Hmmm! is it worst film ever? well sort of, for some of the cast its a shame to see them in such a film but hey if it pays the bills why not, as for the film well. OK cg effects not to bad for such a cheap film ,music is just about OK again for a cheap film, end credits are OK lol<br /><br />BAD to many to list but, cast, acting, sets, script, ending..what the hell ,Drac..........worst Drac EVER!, many more but can not be bothered to put them all down.<br /><br />Idea was OK but needed ten times the budget and more thought and much better lighting and style and change all bad points, i do say however to see this film so you to can say"What the FU%$ was that all about"as the credits run.Also its kind of a must see just to see how bad it is.

Hmmm

!

is

it

worst

film

ever

?

well

sort

of

,

for

some

of

the

cast

its

a

shame

to

see

them

in

such

a

film

but

hey

if

it

pays

the

bills

why

not

,

as

for

the

film

well

.

OK

cg

effects

not

to

bad

for

suc

In [57]:
# Hyperparameters
state_dimensions = [100] #[20, 50, 100, 200, 500]
embedding_dim = 50  # You can choose the dimension of the word embeddings -- Note: this needs to match the model I pick from Glove!!
max_len = 100  # You can adjust this based on the average length of reviews

### ** state dimension, and the max length of the reviews need to be the same you dummy.....

In [58]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [59]:
# If the texts are longer than 500 words, shorten them, so they have a max of 500 words.
def truncate_nested_arrays(array, max_length):
    new_array = [None] * len(array)
    for i in range(len(array)):
        new_array[i] = array[i][:max_length]
    return new_array


In [60]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torchtext

# Tokenize and pad sequences
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
train_texts_tokenized = [tokenizer(review) for review in train_texts]
test_texts_tokenized = [tokenizer(review) for review in test_texts]

# test_index_to_print = 2
# print(train_texts_tokenized[test_index_to_print])
# print(len(train_texts_tokenized[test_index_to_print]))

train_texts_tokenized = truncate_nested_arrays(train_texts_tokenized, max_len)
test_texts_tokenized = truncate_nested_arrays(test_texts_tokenized, max_len)

# print(train_texts_tokenized[test_index_to_print])
# print(len(train_texts_tokenized[test_index_to_print]))


In [61]:
# Build vocabulary
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

vocab = torchtext.vocab.build_vocab_from_iterator(train_texts_tokenized + test_texts_tokenized) # Note: This probably shouldn't include the test_texts, but deal with this later

train_sequences = [torch.tensor([vocab[token] for token in review], dtype=torch.long, device=device)for review in train_texts_tokenized]
test_sequences = [torch.tensor([vocab[token]for token in review], dtype=torch.long, device=device) for review in test_texts_tokenized] # Error thrown here since the test tokens may not exist in the vocabulary

train_padded = nn.utils.rnn.pad_sequence(train_sequences, batch_first=True, padding_value=0)
test_padded = nn.utils.rnn.pad_sequence(test_sequences, batch_first=True, padding_value=0)

In [62]:
print(vocab["i"])
print(vocab["found"])

print(train_texts_tokenized[0])

print(len(train_padded[4]))

print(len(test_padded[5]))

8
212
['i', 'found', 'this', 'film', 'to', 'be', 'a', 'fascinating', 'study', 'of', 'a', 'family', 'in', 'crisis', '.', 'when', 'leo', ',', 'the', 'oldest', 'announces', 'that', 'he', 'is', 'hiv+', 'the', 'reactions', 'of', 'the', 'family', 'members', 'alone', 'and', 'with', 'each', 'other', 'was', 'touching', 'and', 'yet', 'strange', '.', 'i', 'have', 'never', 'seen', 'a', 'family', 'that', 'was', 'as', 'physically', 'demonstrative', 'as', 'this', 'one', 'nor', 'one', 'as', 'likely', 'to', 'shout', 'at', 'each', 'other', '.', 'i', 'didn', "'", 't', 'understand', 'why', 'the', 'family', 'felt', 'that', 'youngest', 'couldn', "'", 't', 'deal', 'with', 'the', 'news', 'but', 'once', 'past', 'that', 'difficult', 'i', 'found', 'this', 'a', 'thoroughly', 'moving', 'film', '.']
100
100


In [63]:
# Download GloVe word embeddings
# You can download from https://nlp.stanford.edu/projects/glove
# I'll assume you've downloaded the 100-dimensional GloVe embeddings

glove_file = word_embedding_file_path
embedding_index = {}
with open(glove_file, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

In [64]:
print(embedding_index.get("123"))
print(embedding_index.get("volptuous"))

[ 0.0034133 -0.14757    1.2649    -0.21882    0.31136    0.094664
  0.21661   -0.56447    0.22246    0.090778  -0.93579   -0.76006
 -0.066726  -0.16403   -0.016907  -1.2052    -0.31426    0.25058
 -0.78974    0.33977   -0.086043   0.10269    0.71572   -0.26135
 -0.37595    0.23748    0.62505   -0.43565   -0.39561   -0.23105
  1.3102     0.22167    0.64756    0.86903    0.85008   -0.32513
  0.84591   -0.12774    0.11723    0.41916   -0.54855   -0.45053
  1.1328     0.30328   -1.0642     0.97721   -0.75989    0.18138
  0.22958   -0.31125  ]
None


In [65]:
# Create embedding matrix
embedding_matrix = np.zeros((len(vocab), embedding_dim))
for word, i in vocab.get_stoi().items():
    # print(word, i)
    embedding_vector = embedding_index.get(word)
    # print(embedding_vector)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_matrix_tensor = torch.tensor(embedding_matrix, device=device)



In [66]:
# Build model
class LSTMTextClassifier(nn.Module):
    def __init__(self, embedding_dim, state_dim):
        super(LSTMTextClassifier, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix_tensor, freeze=True)
        self.rnn = nn.LSTM(input_size=embedding_dim, hidden_size=state_dim, batch_first=True, device=device)
        self.fc = nn.Linear(state_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.long()
        x = self.embedding(x)
        x = x.float()
        output, _ = self.rnn(x)
        output = output[:, -1, :]
        output = self.fc(output)
        output = self.sigmoid(output)
        return output


print_things = False

# Build model
class RNNTextClassifier(nn.Module):
    def __init__(self, embedding_dim, state_dim):
        super(RNNTextClassifier, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix_tensor, freeze=True)
        self.rnn = nn.RNN(embedding_dim, state_dim, batch_first=True, device=device)
        self.fc = nn.Linear(state_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        """
        Args:
            x: Input tensor containing a sequence of tokens
        Return: The output of the current RNN, a list out tensors (a list of binary classifications)
        """
        x = x.long()
        if print_things: print(f"Passed in: {x}")
        vector_embedding = self.embedding(x).float()
        if print_things: print(f"Embedding lookup float: {vector_embedding}")
        rnn_output, _ = self.rnn(vector_embedding)

        if print_things: print(f"Embedding lookup float: {rnn_output}")
        # Option 1: Pooling (e.g., max pooling)
        max_pooling_output, _ = torch.max(rnn_output, 1)
        output = self.fc(max_pooling_output)
        output = self.sigmoid(output)
        if print_things: print(f"Model output: {output}")
        return output

def model_training_loop(model, criterion, optimizer, training_data_loader):

    training_loop_print = False

    loss = None
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for batch in training_data_loader:
            inputs, labels = batch[0], batch[1]
            inputs, labels = inputs.to(device), labels.to(device)
            # if training_loop_print: print(f"Training Inputs: {inputs}")
            if training_loop_print: print(f"Training Label: {labels}")
            optimizer.zero_grad()
            outputs = model(inputs)
            if training_loop_print: print(f"Training Output: {outputs}")


            labels = labels.unsqueeze(1)
            loss = criterion(outputs, labels)
            if training_loop_print: print(f"Training Loss: {loss}")
            if training_loop_print: print(f"----")
            loss.backward()
            optimizer.step()
        print(loss)


In [67]:
batch_size_training = 64
batch_size_testing = 64
learning_rate = 0.001
num_epochs = 5

# Model training loop
results = []
# Convert padded to torch.float32
train_padded = train_padded.float()
test_padded = test_padded.float()

train_label_tensor = torch.tensor(train_labels, dtype=torch.float32, device=device)
test_label_tensor = torch.tensor(test_labels, dtype=torch.float32, device=device)

# Convert to tensor dataset
train_dataset = TensorDataset(train_padded, train_label_tensor)
test_dataset = TensorDataset(test_padded, test_label_tensor)

# Converty the tensor datasets into a dataloader
train_loader = DataLoader(train_dataset, batch_size=batch_size_training, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size_testing, shuffle=True)


for state_dim in state_dimensions:
    for model_type in ["RNN"]: #, "LSTM"]:

        model = None
        if model_type == "RNN":
          model = RNNTextClassifier(embedding_dim, state_dim).to(device)
        elif model_type == "LSTM":
          model = LSTMTextClassifier(embedding_dim, state_dim).to(device)

        # Loss and optimizer
        # criterion = nn.BCELoss() # Cross entropy preffered for classification tasks
        criterion = nn.BCELoss() # Different criterion commonly used for classification tasks
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        model_training_loop(model, criterion, optimizer, train_loader)

        # model_using_iterator(model, criterion, optimizer, train_loader)

        # Evaluation
        model.eval()

        print_test_info = False

        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                # if print_test_info: print(f"Input: {inputs}")
                if print_test_info: print(f"Input Labels: {labels}")
                outputs = model(inputs)
                if print_test_info: print(f"Model Output: {outputs}")
                predicted = (outputs.squeeze() > 0.5).float()
                if print_test_info: print(f"Predicted: {predicted}")
                if print_test_info: print("---")
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total * 100
        results.append((model_type, state_dim, accuracy))

# Display results
print("Results:")
print("| Model | State Dimension | Accuracy |")
print("|-------|------------------|----------|")
for result in results:
    print(f"| {result[0]} | {result[1]} | {result[2]:.4f}% |")

tensor(0.5056, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4136, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4308, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.4395, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.2927, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Results:
| Model | State Dimension | Accuracy |
|-------|------------------|----------|
| RNN | 100 | 77.7520% |
